Cross Validation 2

size =  20247

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.5) & (df['Maximum_Installs'] >= 20000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_20247/CV_df_train_2.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_20247/CV_df_val_2.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(16198, 23)
(4049, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Fortune Teller Khmer,com.khemarasoft.fortuneteller,Lifestyle,4.5,50284,False,Want to find out your future? This app is a mu...,http://khemarasoft.com/app-policy,0,0,...,0,0,0,0,0,0,370,want to find out your future? this app is a mu...,365,76
1,Mrsool | مرسول,com.mrsool,Shopping,4.5,6450386,False,Mrsool is one of the largest delivery platform...,https://s.mrsool.co/privacy_policy.html,0,1,...,0,0,1,1,1,0,1356,mrsool is one of the largest delivery platform...,1310,267


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        2466
Location      2444
Microphone    1465
Contacts      1620
Storage       8174
Phone         2378
SMS             59
Call_Log        35
Calendar       279
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         632
Location       636
Microphone     385
Contacts       430
Storage       2079
Phone          621
SMS             12
Call_Log         8
Calendar        73
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

20247


In [23]:
corpus[0]

['want',
 'find',
 'future',
 'app',
 'must',
 'like',
 'read',
 'fortune',
 'fortune',
 'teller',
 'khmer',
 'collection',
 'khmer',
 'language',
 'simple',
 'way',
 'laughs',
 'fun',
 'simple',
 'use',
 'input',
 'phone',
 'number',
 'name',
 'birthday',
 'etc',
 'application',
 'intended',
 'entertainment',
 'purpose',
 'actually',
 'capable',
 'telling',
 'future']

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        want find future app must like read fortune fo...
1        mrsool one largest delivery platforms kingdom ...
2        hombi social app managing apartment buildings ...
3        oos skin playerpro free color accents pink blu...
4        lokseva academy online platform managing data ...
                               ...                        
16193    time workouts plans hybrid interval timer app ...
16194    ever twitch play loudest gachi want express lo...
16195      synonyms relationships entertainment birds v...
16196    colorful lion theme gives best keyboard backgr...
16197    keep track weekly k run results parkrunner ana...
Name: Clean_Description, Length: 16198, dtype: object

In [29]:
train_sequences

[[37,
  30,
  462,
  1,
  383,
  19,
  142,
  2144,
  2144,
  5661,
  1283,
  167,
  1283,
  45,
  52,
  71,
  123,
  52,
  7,
  335,
  6,
  121,
  264,
  1055,
  57,
  11,
  1534,
  971,
  1059,
  2060,
  5277,
  3530,
  462],
 [12931,
  18,
  1871,
  1041,
  1992,
  2779,
  12931,
  175,
  2442,
  82,
  4708,
  1811,
  84,
  2394,
  896,
  784,
  1041,
  40,
  150,
  23,
  2851,
  1,
  4793,
  491,
  510,
  1149,
  2974,
  663,
  1923,
  1528,
  2779,
  2851,
  3262,
  289,
  5662,
  5350,
  3263,
  7781,
  747,
  781,
  2310,
  12931,
  1041,
  1,
  2443,
  1419,
  12931,
  19,
  4755,
  226,
  1439,
  921,
  303,
  2443,
  571,
  2974,
  16,
  2443,
  371,
  277,
  1496,
  3812,
  2432,
  2422,
  139,
  4261,
  956,
  6384,
  888,
  12931,
  544,
  364,
  12931,
  2160,
  4793,
  491,
  124,
  2161,
  37,
  364,
  997,
  7,
  12931,
  4432,
  6498,
  429,
  303,
  630,
  424,
  768,
  1419,
  303,
  91,
  303,
  168,
  1355,
  303,
  1923,
  2974,
  1149,
  12932,
  226,
  250,
  2

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[   37,    30,   462, ...,     0,     0,     0],
       [12931,    18,  1871, ...,     0,     0,     0],
       [19513,   158,     1, ...,     0,     0,     0],
       ...,
       [ 3620,  2490,   971, ...,     0,     0,     0],
       [  216,   752,     3, ...,     0,     0,     0],
       [  122,   201,   785, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[ 701, 5591, 2791, ...,    0,    0,    0],
       [   1,  141,  159, ...,    0,    0,    0],
       [  37,   59,  766, ...,    0,    0,    0],
       ...,
       [   4,  413,  260, ...,    0,    0,    0],
       [  11, 3698,  872, ...,    0,    0,    0],
       [ 370,   28, 8261, ...,    0,    0,    0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

want find future app must like read fortune fortune teller khmer collection khmer language simple way laughs fun simple use input phone number name birthday etc application intended entertainment purpose actually capable telling future
[37, 30, 462, 1, 383, 19, 142, 2144, 2144, 5661, 1283, 167, 1283, 45, 52, 71, 123, 52, 7, 335, 6, 121, 264, 1055, 57, 11, 1534, 971, 1059, 2060, 5277, 3530, 462]


In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 66960


In [36]:
word_index

{'app': 1,
 'keyboard': 2,
 'theme': 3,
 'free': 4,
 'wallpaper': 5,
 'phone': 6,
 'use': 7,
 'wallpapers': 8,
 'new': 9,
 'features': 10,
 'application': 11,
 'get': 12,
 'time': 13,
 'make': 14,
 'android': 15,
 'also': 16,
 'launcher': 17,
 'one': 18,
 'like': 19,
 'download': 20,
 'screen': 21,
 'hd': 22,
 'best': 23,
 'learn': 24,
 'english': 25,
 'live': 26,
 'easy': 27,
 'us': 28,
 'themes': 29,
 'find': 30,
 'love': 31,
 'help': 32,
 'share': 33,
 'mobile': 34,
 'please': 35,
 'set': 36,
 'want': 37,
 'support': 38,
 'device': 39,
 'apps': 40,
 'words': 41,
 'home': 42,
 'need': 43,
 'day': 44,
 'language': 45,
 'enjoy': 46,
 'using': 47,
 'images': 48,
 'available': 49,
 'many': 50,
 'different': 51,
 'simple': 52,
 'note': 53,
 'beautiful': 54,
 'add': 55,
 'text': 56,
 'etc': 57,
 'information': 58,
 'apply': 59,
 'background': 60,
 'access': 61,
 'daily': 62,
 'learning': 63,
 'friends': 64,
 'every': 65,
 'life': 66,
 'install': 67,
 'you': 68,
 'easily': 69,
 'stickers': 

In [37]:
word_index["reason"]

2489

In [38]:
print(validation_sequences[0])

[701, 5591, 2791, 2, 3, 9, 89, 430, 123, 233, 49, 12, 701, 5591, 2791, 2, 3, 1519, 1252, 14, 6, 156, 1472, 502, 1189, 701, 5591, 2791, 2, 3, 544, 6, 2081, 399, 2002, 3562, 701, 5591, 2791, 2, 3, 1362, 134, 82, 20, 67, 701, 5591, 2791, 2, 3, 4, 129, 85, 82, 2812, 2775, 2391, 73, 200, 701, 5591, 2791, 2, 3, 589, 193, 6, 2080, 2307, 701, 5591, 2791, 2, 3, 2213, 31, 701, 5591, 2791, 2, 3, 7, 701, 5591, 2791, 2, 3, 53, 35, 20, 791, 2, 150, 20, 701, 5591, 2791, 2, 3, 81, 79, 98, 147, 91, 59, 135, 503, 266, 701, 5591, 2791, 2, 3, 811, 332, 490, 701, 5591, 2791, 2, 3, 164, 214, 186, 244, 126, 253, 244, 101, 36, 22, 8, 7, 41, 716, 14, 343, 337, 10, 93, 317, 1020, 134, 210, 83, 698, 203, 615, 461, 698, 176, 335, 323, 1309, 534, 1365, 1896, 2, 405, 9, 10, 1185, 134, 2841, 82, 2, 396, 661, 427, 78, 38, 114, 434, 591, 2, 399, 2, 15, 296, 156, 15, 29, 1545, 2, 3, 1010, 2, 132, 362, 4, 157, 156, 418, 2, 22, 8, 392, 6, 60, 19, 359, 22, 5, 104, 22, 5, 165, 22, 5, 874, 608, 669, 22, 8, 1300, 29, 89, 29,

In [39]:
word_index["listen"]

304

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.51651001,  0.090911  , -0.15025   , ...,  0.17309999,
         0.094735  , -0.62935001],
       ...,
       [ 0.37533   , -0.32378   ,  0.49807   , ..., -0.27970001,
         0.27811   ,  0.10715   ],
       [-0.16889   , -0.16912   ,  0.47323   , ..., -0.091788  ,
        -0.14264999, -0.68147999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2489

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(16198, 600)
(16198, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(4049, 600)
(4049, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

16198
9
{0: 0.7298368928539245, 1: 0.7364066193853428, 2: 1.228517254455821, 3: 1.1109739368998628, 4: 0.22018323682135768, 5: 0.7568451546584432, 6: 30.504708097928436, 7: 51.422222222222224, 8: 6.450816407805655}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    20088300    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    20088300    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    20088300    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
507/507 [==============================] - 109s 192ms/step - loss: 0.1936 - accuracy: 0.2811 - val_loss: 0.2490 - val_accuracy: 0.3025
Epoch 2/10
507/507 [==============================] - 96s 190ms/step - loss: 0.1556 - accuracy: 0.2859 - val_loss: 0.2472 - val_accuracy: 0.2843
Epoch 3/10
507/507 [==============================] - 96s 189ms/step - loss: 0.1354 - accuracy: 0.2892 - val_loss: 0.2498 - val_accuracy: 0.2946
Epoch 4/10
507/507 [==============================] - 96s 189ms/step - loss: 0.1150 - accuracy: 0.2928 - val_loss: 0.2203 - val_accuracy: 0.2897
Epoch 5/10
507/507 [==============================] - 96s 189ms/step - loss: 0.1011 - accuracy: 0.2976 - val_loss: 0.2235 - val_accuracy: 0.3045
Epoch 6/10
507/507 [==============================] - 96s 189ms/step - loss: 0.0876 - accuracy: 0.3061 - val_loss: 0.2247 - val_accuracy: 0.3011
Epoch 7/10
507/507 [==============================] - 96s 189ms/step - loss: 0.0700 - accuracy: 0.3188 - val_loss: 0.2444 - val_a

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2280, 1799, 30, 23, 2280, 9037, 459, 757, 2066, 1, 1, 204, 13, 23, 459, 2280, 1159, 459, 9037, 116, 459, 2280, 167, 23, 459, 2280, 1159, 459, 9037, 19, 16180, 17788, 410, 16, 204, 432, 459, 2280, 1710, 459, 2280, 459, 8274, 116, 459, 739, 459, 739, 1, 204, 23, 502, 2280, 1159, 9037, 212, 1

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[6.9267931e-04, 7.8260904e-04, 2.0830869e-04, 1.9481219e-04,
        4.0952760e-01, 2.2048005e-03, 3.7508780e-05, 9.7737620e-07,
        2.2887269e-05],
       [3.3385912e-04, 6.3036510e-04, 5.0418617e-05, 1.6758402e-04,
        6.5596518e-03, 6.5913826e-04, 2.7043070e-04, 2.8153283e-06,
        2.2588536e-04],
       [5.9151101e-01, 1.2035905e-02, 2.0766604e-01, 2.3052277e-01,
        9.8940408e-01, 2.1509424e-01, 2.9962910e-03, 7.3718157e-04,
        5.6904913e-03],
       [1.5562872e-03, 1.1876209e-03, 6.4293599e-05, 8.3280960e-04,
        2.1533559e-01, 2.9460099e-03, 1.7463174e-04, 3.3652273e-06,
        1.7636686e-04],
       [2.9708855e-03, 6.6629355e-04, 8.6363325e-05, 1.0622141e-02,
        5.6245900e-03, 7.0188870e-03, 8.9149466e-03, 1.6179137e-04,
        2.7214251e-03],
       [3.9317289e-01, 6.6754986e-03, 8.6031854e-02, 8.0000870e-02,
        9.9472213e-01, 9.5534489e-02, 9.9612505e-04, 1.4454599e-04,
        7.1952445e-04],
       [7.5883150e-01, 1.6626976e-01, 9.

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[6.9267931e-04 7.8260904e-04 2.0830869e-04 ... 3.7508780e-05
  9.7737620e-07 2.2887269e-05]
 [3.3385912e-04 6.3036510e-04 5.0418617e-05 ... 2.7043070e-04
  2.8153283e-06 2.2588536e-04]
 [5.9151101e-01 1.2035905e-02 2.0766604e-01 ... 2.9962910e-03
  7.3718157e-04 5.6904913e-03]
 ...
 [9.2542750e-01 2.8388955e-02 9.8281616e-01 ... 3.1294881e-03
  1.4330066e-03 6.5447107e-02]
 [1.2444924e-02 9.8165131e-01 9.9944872e-01 ... 2.5101787e-07
  2.1734500e-07 3.3693048e-04]
 [2.9177120e-04 8.4101754e-05 2.9004080e-04 ... 4.0233124e-04
  5.7607017e-06 5.5022736e-04]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.654 with F-Score: 0.6772
permission  1
Length of sequence: 10000
Best Threshold: 0.1077 with F-Score: 0.572
permission  2
Length of sequence: 10000
Best Threshold: 0.3463 with F-Score: 0.5628
permission  3
Length of sequence: 10000
Best Threshold: 0.33 with F-Score: 0.5267
permission  4
Length of sequence: 10000
Best Threshold: 0.4754 with F-Score: 0.8013
permission  5
Length of sequence: 10000
Best Threshold: 0.0822 with F-Score: 0.5019
permission  6
Length of sequence: 10000
Best Threshold: 0.0257 with F-Score: 0.0909
permission  7
Length of sequence: 10000
Best Threshold: 0.0472 with F-Score: 0.3333
permission  8
Length of sequence: 10000
Best Threshold: 0.1236 with F-Score: 0.3611
-------------------------------------
optimal threshold tuning for f-score
[0.654, 0.1077, 0.3463, 0.33, 0.4754, 0.0822, 0.0257, 0.0472, 0.1236]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.2175000011920929 with G-Mean: 0.8352
FPR: 0.1266, TPR: 0.7987
permission  1
Best Threshold: 0.021700000390410423 with G-Mean: 0.7745
FPR: 0.1896, TPR: 0.7402
permission  2
Best Threshold: 0.012500000186264515 with G-Mean: 0.8227
FPR: 0.1848, TPR: 0.8303
permission  3
Best Threshold: 0.027899999171495438 with G-Mean: 0.8089
FPR: 0.2228, TPR: 0.842
permission  4
Best Threshold: 0.6891999840736389 with G-Mean: 0.7852
FPR: 0.2259, TPR: 0.7964
permission  5
Best Threshold: 0.028999999165534973 with G-Mean: 0.7598
FPR: 0.2412, TPR: 0.7607
permission  6
Best Threshold: 0.002400000113993883 with G-Mean: 0.8883
FPR: 0.2109, TPR: 1.0
permission  7
Best Threshold: 0.0003000000142492354 with G-Mean: 0.8958
FPR: 0.1975, TPR: 1.0
permission  8
Best Threshold: 0.004100000020116568 with G-Mean: 0.8168
FPR: 0.21, TPR: 0.8444
-------------------------------------
ROC curve with G-mean threshold tuning
[0.2175, 0.0217, 0.0125, 0.0279, 0.6892, 0.029, 0.0024, 0.0003, 0.0041]

## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV2_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6772
  Location  : 0.5720
  Microphone: 0.5628
  Contacts  : 0.5267
  Storage   : 0.8013
  Phone     : 0.5019
  SMS       : 0.0909
  Call_Log  : 0.3333
  Calendar  : 0.3611

  Average F1 score: 0.4919


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV2_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8986
  Location  : 0.8631
  Microphone: 0.9187
  Contacts  : 0.8927
  Storage   : 0.7818
  Phone     : 0.8300
  SMS       : 0.9784
  Call_Log  : 0.9974
  Calendar  : 0.9801

  Average F1 (micro) score: 0.9045


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV2_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8360
  Location  : 0.7746
  Microphone: 0.8216
  Contacts  : 0.8096
  Storage   : 0.7848
  Phone     : 0.7590
  SMS       : 0.8497
  Call_Log  : 0.8225
  Calendar  : 0.8119

  Average ROC_AUC score: 0.8078


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.2909 with F-Score: 0.4644
permission  1
Length of sequence: 10000
Best Threshold: 0.0481 with F-Score: 0.5509
permission  2
Length of sequence: 10000
Best Threshold: 0.1485 with F-Score: 0.5291
permission  3
Length of sequence: 10000
Best Threshold: 0.0692 with F-Score: 0.614
permission  4
Length of sequence: 10000
Best Threshold: 0.049 with F-Score: 0.5975
permission  5
Length of sequence: 10000
Best Threshold: 0.1756 with F-Score: 0.3712
permission  6
Length of sequence: 10000
Best Threshold: 0.0159 with F-Score: 0.5653
permission  7
Length of sequence: 10000
Best Threshold: 0.0068 with F-Score: 0.3237
permission  8
Length of sequence: 10000
Best Threshold: 0.0588 with F-Score: 0.5263
-------------------------------------
optimal threshold tuning for f-score
[0.2909, 0.0481, 0.1485, 0.0692, 0.049, 0.1756, 0.0159, 0.0068, 0.0588]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.26820001006126404 with G-Mean: 0.7128
FPR: 0.2346, TPR: 0.6638
permission  1
Best Threshold: 0.014800000004470348 with G-Mean: 0.7284
FPR: 0.3279, TPR: 0.7895
permission  2
Best Threshold: 0.04170000180602074 with G-Mean: 0.7769
FPR: 0.1651, TPR: 0.723
permission  3
Best Threshold: 0.0697999969124794 with G-Mean: 0.7211
FPR: 0.2727, TPR: 0.7149
permission  4
Best Threshold: 0.7049000263214111 with G-Mean: 0.5949
FPR: 0.4098, TPR: 0.5997
permission  5
Best Threshold: 0.11710000038146973 with G-Mean: 0.7414
FPR: 0.1862, TPR: 0.6754
permission  6
Best Threshold: 0.004699999932199717 with G-Mean: 0.8044
FPR: 0.2072, TPR: 0.816
permission  7
Best Threshold: 0.0003000000142492354 with G-Mean: 0.7591
FPR: 0.3642, TPR: 0.9062
permission  8
Best Threshold: 0.01590000092983246 with G-Mean: 0.8568
FPR: 0.1702, TPR: 0.8846
-------------------------------------
ROC curve with G-mean threshold tuning
[0.2682, 0.0148, 0.0417, 0.0698, 0.7049, 0.1171, 0.0047, 0.0003, 0.0

#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV2_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4644
  Location  : 0.5509
  Microphone: 0.5291
  Contacts  : 0.6140
  Storage   : 0.5975
  Phone     : 0.3712
  SMS       : 0.5653
  Call_Log  : 0.3237
  Calendar  : 0.5263

  Average F1 score: 0.5047


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV2_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.7553
  Location  : 0.7786
  Microphone: 0.8854
  Contacts  : 0.7235
  Storage   : 0.4844
  Phone     : 0.8395
  SMS       : 0.8706
  Call_Log  : 0.8847
  Calendar  : 0.9173

  Average F1 (micro) score: 0.7933


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV2_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7124
  Location  : 0.7308
  Microphone: 0.7756
  Contacts  : 0.7211
  Storage   : 0.5941
  Phone     : 0.7403
  SMS       : 0.8044
  Call_Log  : 0.7474
  Calendar  : 0.8524

  Average ROC_AUC score: 0.7420
